https://fred.stlouisfed.org/docs/api/fred/

https://medium.com/swlh/pandas-datareader-federal-reserve-economic-data-fred-a360c5795013

# Return Predictability and Market-Timing: A One-Month Model

By: Blair Hull, Xiao Qiao, Petra Bakosova <br>
2017 <br>

We propose a one-month market-timing model constructed from 15 diverse variables. We use weighted least squares with stepwise variable selection to build a predictive model for the one-month- ahead market excess returns. From our statistical model, we transform our forecasts into investable positions to build a market-timing strategy. From 2003 to 2017, our strategy results in 16.6% annual returns with a 0.92 Sharpe ratio and a 20.3% maximum drawdown, whereas the S&P 500 has annual returns of 10%, a 0.46 Sharpe ratio, and a maximum drawdown of 55.2%. When our one-month model is used in conjunction with Hull and Qiao’s (2017) six-month model, the Sharpe ratio of the combined strategy exceeds the individual model Sharpe ratios. The combined model has 15% annual returns, a Sharpe ratio of 1.12, and a maximum drawdown of 14%. We publish forecasts from our one-month model in our Daily Report.

## Data

1. Change in Inflation (UI): Monthly change in the inflation rate net of the change in the risk-free rate. Inflation is calculated as the percentage change in the Consumer Price Index from ALFRED and the risk-free rate is the three-month Treasury bill rate from Bloomberg. Positive change in inflation is associated with higher future market excess returns. Our variable is similar to “unexpected inflation” in Chen et al. (1986), calculated as the difference between inflation and its expectation using the Fama and Gibbons (1984) method.

2. Industrial Production (IP): Monthly change in the industrial production index published by ALFRED. IP serves as a leading indicator for the real economy, and is positively associated with future market excess returns (Chen et al., 1986).

3. Credit Risk Premium (CRP): The difference between the BAA and AAA corporate bond yields, also known as the default spread, obtained from Bloomberg. Fama and French (1989) demonstrate that a higher CRP is associated with higher future market excess returns.
4. Slope of the Interest Rate Term Structure (STS): The difference between the 10-year Treasury note and the three-month Treasury bill yields from Bloomberg. This quantity is sometimes called the term spread. STS is positively associated with future market excess returns (Fama and French, 1989).

5. Commodity Price (CP): We use the monthly change in the S&P GSCI index to track the movements in oil price, since the fluctuations in the GSCI are predominantly driven by oil price changes. Historically commodity prices and stock prices have been negatively correlated. We expect an increase in CP to be associated with lower future market excess returns (Black et al., 2014; Casassus and Higuera, 2011).

6. Housing Starts (HS): Monthly difference in the housing starts index from ALFRED. HS is another leading indicator for market cycles, and should be positively correlated with future market excess returns (Flannery and Protopapadakis, 2002).

7. Exchange Rate (EVUSD): Monthly change in the U.S. Dollar Index (DXY) from Bloomberg. Higher dollar exchange predicts lower future market excess returns (Bahmani-Oskooee and Sohrabian, 1992).

8. FRB Loan Officer Survey (LOAN): We compute what fraction of banks have been tightening lending conditions compared to the previous quarter as a proxy for the change in lending conditions among U.S. banks. This variable is from the Federal Reserve Board (Lown et al., 2000). An increase in LOAN indicates credit tightening, which we expect to be associated with lower future economic activity and lower future excess returns.

9. Delinquencies (DL): Annual change in delinquencies from the Federal Reserve Board. Similar to LOAN, this is another variable that we use to capture the macroeconomicconditions of banks, and complements the information contained in LOAN. Higher DL forecasts lower future excess returns (Lown et al., 2000).

10. New Orders New Shipments (NONS): The value of new orders and new shipments excluding defense and aircraft. We exclude defense and aircraft spending to focus on the core capital goods which reveal the underlying trend in real economic activity. Higher NONS is associated with lower future market excess returns (Jones and Tuzel, 2012).

11. Baltic Dry Index (BD): Monthly percent change in the Baltic Dry Index from Bloomberg. We use BD to track global shipping as one measurement of macroeconomic conditions. BD is a leading indicator of economic cycles, and is positively correlated with future excess returns. (Bakshi, Panayotov, and Skoulakis, 2011)

12. National Association of Purchasing Managers (NAPM) Survey: The difference between the manufacturing survey new orders and the prices paid indexes from ALFRED. An increase in NAPM signifies an expansion which is a leading positive indicator for stocks.

13. Change in Unemployment Rate (UR): Monthly change in the unemployment rate from ALFRED. Increases in the unemployment rate indicate deteriorating macroeconomic conditions in the future and are associated with low future market excess returns (Flannery and Protopapadakis, 2002).

14. Momentum (MOM): Monthly percent change in the price return of the S&P 500. Moskowitz et al. (2012) suggests past 12-month returns are a positive signal for next month’s returns. We use past one-month return to capture shorter trend in the data. MOM positively predicts future excess returns.

15. PCA of Price Ratios (PRC): We include the first principal component of the cyclically adjusted price-to-earnings ratio (CAPE), cyclically adjusted price to total yield (dividends plus buybacks), and the price-to-book ratio. This composite variable is negatively correlated with future market excess returns. Shiller (2000) proposed the first use of CAPE, and Hull and Qiao (2017) reduce the dimension of price-based variables through PCA.

In [10]:
import pandas as pd
import pandas_datareader as pdr
import datetime
import numpy as np

# 1 Inflation

Change in Inflation (UI): Monthly change in the inflation rate net of the change in the risk-free rate. Inflation is calculated as the percentage change in the Consumer Price Index from ALFRED and the risk-free rate is the three-month Treasury bill rate from Bloomberg. Positive change in inflation is associated with higher future market excess returns. Our variable is similar to “unexpected inflation” in Chen et al. (1986), calculated as the difference between inflation and its expectation using the Fama and Gibbons (1984) method.

https://alfred.stlouisfed.org/series?seid=CPIAUCNS

https://fred.stlouisfed.org/series/DTB3

In [11]:
start = datetime.datetime (2005, 1, 1)
end = datetime.datetime (2022, 3, 7)

inflation_df = pdr.DataReader('CPIAUCNS', 'fred', start, end)


In [16]:
# np.log(inflation_df.CPIAUCNS)
inflation_df['cpi_pct_change'] = np.log(inflation_df.CPIAUCNS).diff()

In [17]:
inflation_df

,CPIAUCNS,cpi_pct_change
DATE,,
2005-01-01,190.700,NaN
2005-02-01,191.800,0.005752
2005-03-01,193.300,0.007790
2005-04-01,194.600,0.006703
2005-05-01,194.400,-0.001028
...,...,...
2021-09-01,274.310,0.002712
2021-10-01,276.589,0.008274
2021-11-01,277.948,0.004901


In [7]:
start = datetime.datetime (2005, 1, 1)
end = datetime.datetime (2022, 3, 7)

risk_free_df = pdr.DataReader('DTB3', 'fred', start, end)
risk_free_df

,DTB3
DATE,
2005-01-03,2.29
2005-01-04,2.30
2005-01-05,2.29
2005-01-06,2.27
2005-01-07,2.28
...,...
2022-03-01,0.32
2022-03-02,0.33
2022-03-03,0.37


# 2 Industrial Production

Industrial Production (IP): Monthly change in the industrial production index published by ALFRED. IP serves as a leading indicator for the real economy, and is positively associated with future market excess returns (Chen et al., 1986).

https://alfred.stlouisfed.org/series?seid=INDPRO

In [18]:
start = datetime.datetime (2005, 1, 1)
end = datetime.datetime (2022, 3, 7)

ip_df = pdr.DataReader('INDPRO', 'fred', start, end)
ip_df

,INDPRO
DATE,
2005-01-01,96.1164
2005-02-01,96.8200
2005-03-01,96.6725
2005-04-01,96.8638
2005-05-01,96.9622
...,...
2021-09-01,99.8440
2021-10-01,101.2341
2021-11-01,102.1319


# 3 Credit Risk Premium

Credit Risk Premium (CRP): The difference between the BAA and AAA corporate bond yields, also known as the default spread, obtained from Bloomberg. Fama and French (1989) demonstrate that a higher CRP is associated with higher future market excess returns.

https://fred.stlouisfed.org/series/BAA

https://fred.stlouisfed.org/series/AAA

In [19]:
baa_df = pdr.DataReader('BAA', 'fred', start, end)
baa_df

,BAA
DATE,
2005-01-01,6.02
2005-02-01,5.82
2005-03-01,6.06
2005-04-01,6.05
2005-05-01,6.01
...,...
2021-10-01,3.35
2021-11-01,3.28
2021-12-01,3.30


In [20]:
aaa_df = pdr.DataReader('AAA', 'fred', start, end)
aaa_df

,AAA
DATE,
2005-01-01,5.36
2005-02-01,5.20
2005-03-01,5.40
2005-04-01,5.33
2005-05-01,5.15
...,...
2021-10-01,2.68
2021-11-01,2.62
2021-12-01,2.65


# 4 Slope of the Interest Rate Term Structure

Slope of the Interest Rate Term Structure (STS): The difference between the 10-year Treasury note and the three-month Treasury bill yields from Bloomberg. This quantity is sometimes called the term spread. STS is positively associated with future market excess returns (Fama and French, 1989).

https://fred.stlouisfed.org/series/DGS10

https://fred.stlouisfed.org/series/DTB3

# 5 Commodity Price 

Commodity Price (CP): We use the monthly change in the S&P GSCI index to track the movements in oil price, since the fluctuations in the GSCI are predominantly driven by oil price changes. Historically commodity prices and stock prices have been negatively correlated. We expect an increase in CP to be associated with lower future market excess returns (Black et al., 2014; Casassus and Higuera, 2011).

https://tradingeconomics.com/commodity/gsci

# 6 Housing Starts

Housing Starts (HS): Monthly difference in the housing starts index from ALFRED. HS is another leading indicator for market cycles, and should be positively correlated with future market excess returns (Flannery and Protopapadakis, 2002).

https://fred.stlouisfed.org/series/HOUSTNSA

# 7 Exchange Rate

Exchange Rate (EVUSD): Monthly change in the U.S. Dollar Index (DXY) from Bloomberg. Higher dollar exchange predicts lower future market excess returns (Bahmani-Oskooee and Sohrabian, 1992).




# 8 FRB Loan Officer Survey

FRB Loan Officer Survey (LOAN): We compute what fraction of banks have been tightening lending conditions compared to the previous quarter as a proxy for the change in lending conditions among U.S. banks. This variable is from the Federal Reserve Board (Lown et al., 2000). An increase in LOAN indicates credit tightening, which we expect to be associated with lower future economic activity and lower future excess returns.

https://fred.stlouisfed.org/categories/32239

# 9 Delinquencies

Delinquencies (DL): Annual change in delinquencies from the Federal Reserve Board. Similar to LOAN, this is another variable that we use to capture the macroeconomicconditions of banks, and complements the information contained in LOAN. Higher DL forecasts lower future excess returns (Lown et al., 2000).

https://www.federalreserve.gov/releases/chargeoff/delallnsa.htm

# 10 New Orders New Shipments

New Orders New Shipments (NONS): The value of new orders and new shipments excluding defense and aircraft. We exclude defense and aircraft spending to focus on the core capital goods which reveal the underlying trend in real economic activity. Higher NONS is associated with lower future market excess returns (Jones and Tuzel, 2012).

https://fred.stlouisfed.org/series/DGORDER
https://fred.stlouisfed.org/series/NEWORDER

https://fred.stlouisfed.org/series/AMTMVS

# 11 Baltic Dry Index

Baltic Dry Index (BD): Monthly percent change in the Baltic Dry Index from Bloomberg. We use BD to track global shipping as one measurement of macroeconomic conditions. BD is a leading indicator of economic cycles, and is positively correlated with future excess returns. (Bakshi, Panayotov, and Skoulakis, 2011)

https://tradingeconomics.com/commodity/baltic

# 12 National Association of Purchasing Managers

National Association of Purchasing Managers (NAPM) Survey: The difference between the manufacturing survey new orders and the prices paid indexes from ALFRED. An increase in NAPM signifies an expansion which is a leading positive indicator for stocks.



# 13 Change in Unemployment Rate

Change in Unemployment Rate (UR): Monthly change in the unemployment rate from ALFRED. Increases in the unemployment rate indicate deteriorating macroeconomic conditions in the future and are associated with low future market excess returns (Flannery and Protopapadakis, 2002).

https://alfred.stlouisfed.org/series?seid=UNRATENSA

# 14 Momentum

Momentum (MOM): Monthly percent change in the price return of the S&P 500. Moskowitz et al. (2012) suggests past 12-month returns are a positive signal for next month’s returns. We use past one-month return to capture shorter trend in the data. MOM positively predicts future excess returns.



# 15 PCA of Price Ratios

PCA of Price Ratios (PRC): We include the first principal component of the cyclically adjusted price-to-earnings ratio (CAPE), cyclically adjusted price to total yield (dividends plus buybacks), and the price-to-book ratio. This composite variable is negatively correlated with future market excess returns. Shiller (2000) proposed the first use of CAPE, and Hull and Qiao (2017) reduce the dimension of price-based variables through PCA.

https://data.nasdaq.com/data/MULTPL/SHILLER_PE_RATIO_MONTH-shiller-pe-ratio-by-month